#### 国内模型基本一致
 - swiglu  貌似是为了将矩阵变换后计算，本质上和 silu 没有区别，就是为了迎合国内某些模型的架构中维度扩展的问题

 - torch.chunk(input, chunk, dim) 用法
   - a, b = t.chunk(t.randn(3,10,200), 2, dim=-1)
   - a.shape = [3, 10, 100], b.shape = [3, 10, 100]
 - w = torch.nn.Linear(input_size, output_size)
   - w.weight.shape = [output_size, input_size] 满足 x 与 w 的转置乘积

In [1]:
import torch as t
import torch.nn.functional as F


In [2]:
input_size = 20
output_size = 100

In [4]:
linear_combine = t.nn.Linear(in_features=input_size, out_features=output_size*2, bias=False)

In [5]:
random_input = t.randn(3, 10, 20)

In [6]:
def swiglu(x):
    x = t.chunk(x, 2, dim=-1)
    return F.silu(x[0]) * x[1]

output1 = linear_combine(random_input)
output1 = swiglu(output1)
output1.shape

torch.Size([3, 10, 100])

In [13]:
linear_1 = t.nn.Linear(in_features=input_size, out_features=output_size, bias=False)
linear_2 = t.nn.Linear(in_features=input_size, out_features=output_size, bias=False)

# linear_combine.weight.shape = [output_size, input_size]
chunk_weight = t.chunk(linear_combine.weight, 2, dim=0)
linear_1.weight = t.nn.Parameter(chunk_weight[0])
linear_2.weight = t.nn.Parameter(chunk_weight[1])


In [14]:
output2 = F.silu(linear_1(random_input)) * linear_2(random_input)

In [15]:
t.allclose(output1, output2)

True